In [2]:
import numpy as np
import tensorflow as tf
import math

from tensorflow.contrib.integrate.python.ops import odes
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import errors_impl
from tensorflow.python.framework import tensor_shape
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.platform import test

In [2]:
NUM_CORES = 7  # Choose how many cores to use.
sess = tf.Session(
    config=tf.ConfigProto(inter_op_parallelism_threads=NUM_CORES,
                   intra_op_parallelism_threads=NUM_CORES))

In [16]:
# Test odeint by an exponential function:
#   dy / dt = y,  y(0) = 1.0.
# Its analytical solution is y = exp(t).
func = lambda y, t: y
y0 = constant_op.constant(1.0, dtype=dtypes.float64)
t = np.linspace(0.0, 1.0, 11)
y_solved = odes.odeint(func, y0, t)

with tf.Session() as sess:
    y_solved = sess.run(y_solved)
y_true = np.exp(t)

In [19]:
# Test a complex, linear ODE:
#   dy / dt = k * y,  y(0) = 1.0.
# Its analytical solution is y = exp(k * t).
k = 1j - 0.1
func = lambda y, t: k * y
t = np.linspace(0.0, 1.0, 11)
y_solved = odes.odeint(func, 1.0 + 0.0j, t)
with tf.Session() as sess:
    y_solved = sess.run(y_solved)
y_true = np.exp(k * t)

In [ ]:
# Solve the 2D linear differential equation:
#   dy1 / dt = 3.0 * y1 + 4.0 * y2,
#   dy2 / dt = -4.0 * y1 + 3.0 * y2,
#   y1(0) = 0.0,
#   y2(0) = 1.0.
# Its analytical solution is
#   y1 = sin(4.0 * t) * exp(3.0 * t),
#   y2 = cos(4.0 * t) * exp(3.0 * t).
matrix = constant_op.constant(
[[3.0, 4.0], [-4.0, 3.0]], dtype=dtypes.float64)
func = lambda y, t: math_ops.matmul(matrix, y)

y0 = constant_op.constant([[0.0], [1.0]], dtype=dtypes.float64)
t = np.linspace(0.0, 2.0, 10000000)

y_solved = odes.odeint(func, y0, t)

y_solved = sess.run(y_solved)

y_true = np.zeros((len(t), 2, 1))
y_true[:, 0, 0] = np.sin(4.0 * t) * np.exp(3.0 * t)
y_true[:, 1, 0] = np.cos(4.0 * t) * np.exp(3.0 * t)

In [8]:
h = 0.1
n = int(1.0/0.1)
x = np.linspace(0, 1.0, n)
y = np.zeros(n)
z = np.zeros(n)

y[0] = 3
z[0] = 3

# F_xy = @(t,r) 3.*exp(-t)-0.4*r
F_xyz = lambda x, y, z: z
G_xyz = lambda x, y, z: 6 * y - z

for i in range(len(x) - 1):
    k1 = F_xyz(x[i], y[i], z[i])
    L1 = G_xyz(x[i], y[i], z[i])
    k2 = F_xyz(x[i] + .5 * h, y[i] + .5 * h * k1, z[i] + .5 * h * L1)
    L2 = G_xyz(x[i] + .5 * h, y[i] + .5 * h * k1, z[i] + .5 * h * L1)
    k3 = F_xyz(x[i] + .5 * h, y[i] + .5 * h * k2, z[i] + .5 * h * L2)
    L3 = G_xyz(x[i] + .5 * h, y[i] + .5 * h * k2, z[i] + .5 * h * L2)
    k4 = F_xyz(x[i] + h, y[i] + h * k3, z[i] + h * L3)
    L4 = F_xyz(x[i] + h, y[i] + h * k3, z[i] + h * L3)
    
    y[i + 1] = y[i] + (1/6) * (k1 + 2 * k2 + 2 * k3 + k4) * h
    z[i + 1] = z[i] + (1/6) * (L1 + 2 * L2 + 2 * L3 + L4) * h

print(x)
print(y)
print(z)

[ 0.          0.11111111  0.22222222  0.33333333  0.44444444  0.55555556
  0.66666667  0.77777778  0.88888889  1.        ]
[  3.           3.3758625    3.89175666   4.56370441   5.41428888
   6.47332773   7.77885504   9.3784398   11.33088305  13.70835194]
[  3.           4.3428625    5.80991801   7.46456912   9.37523182
  11.61834286  14.28156234  17.4673145   21.2968176   25.91476706]
